In [1]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
sys.path.append(r"C:\Users\rowe\OneDrive - Stichting Deltares\Documents\GitHub\coclicodata\src")
import geojson
import xarray as xr
from coclicodata.etl.cloud_utils import (
    p_drive,
    dataset_to_google_cloud,
    dataset_from_google_cloud,
    geojson_to_mapbox,
    load_env_variables,
    load_google_credentials,
)
from coclicodata.etl.extract import (
    clear_zarr_information,
    get_geojson,
    get_mapbox_url,
    zero_terminated_bytes_as_str,
)
from coclicodata.coclico_stac.utils import (
    get_dimension_dot_product,
    get_dimension_values,
    get_mapbox_item_id,
    rm_special_characters,
)


In [2]:
# ds=xr.open_zarr(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards\01_Present\Extreme_Wave_Height\ERA5_Hourly_2004-2014.zarr")
#ds=xr.open_zarr(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\03_Vulnerability\GDP\GDP_PPP_CAP_final.zarr")
ds=xr.open_zarr(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards\02_Future\Extreme_Wave_Height\GCM_Projection_2040-2050\EC-EARTH_WW3_3Hourly_2040-2050.zarr")
ds

<xarray.Dataset>
Dimensions:    (time: 32144, latitude: 361, longitude: 720)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Data variables:
    hs_1_26    (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85    (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [3]:
new_hs_1_26= ds["hs_1_26"].values.reshape(32144, -1)
new_hs_5_85= ds["hs_5_85"].values.reshape(32144, -1)

MemoryError: Unable to allocate 1015. KiB for an array with shape (1, 361, 720) and data type float32

In [ ]:
#Renaming dims to lon-lat for geojson mapbox
ds = ds.rename_dims({"latitude": "lat","longitude": "lon"})
# 
#Adding station for geojson
ds=ds.expand_dims(dim={"nstations": range(len(ds["latitude"])*len(ds["longitude"]))})
ds=ds.rename_vars( {"nstations": "stations" })
ds

<xarray.Dataset>
Dimensions:    (nstations: 259920, time: 32144, lat: 361, lon: 720)
Coordinates:
  * stations   (nstations) int32 0 1 2 3 4 ... 259916 259917 259918 259919
  * latitude   (lat) float32 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * longitude  (lon) float32 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Dimensions without coordinates: nstations, lat, lon
Data variables:
    hs_1_26    (nstations, time, lat, lon) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
    hs_5_85    (nstations, time, lat, lon) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [ ]:
# ds=
# ds=ds.rename_vars({"latitude": "lat","longitude": "lon"})

In [ ]:
ds = zero_terminated_bytes_as_str(ds)

# remove characters that cause problems in the frontend.
ADDITIONAL_DIMENSIONS = ["time"] 

ds = rm_special_characters(
    ds=ds, dimensions_to_check=ADDITIONAL_DIMENSIONS, characters=["%"]
)
ds

<xarray.Dataset>
Dimensions:    (nstations: 259920, time: 32144, lat: 361, lon: 720)
Coordinates:
  * stations   (nstations) int32 0 1 2 3 4 ... 259916 259917 259918 259919
  * latitude   (lat) float32 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * longitude  (lon) float32 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Dimensions without coordinates: nstations, lat, lon
Data variables:
    hs_1_26    (nstations, time, lat, lon) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
    hs_5_85    (nstations, time, lat, lon) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [ ]:
a=ds.drop_dims(['lon', 'lat'], errors='ignore')
a

<xarray.Dataset>
Dimensions:   (nstations: 259920, time: 32144)
Coordinates:
  * stations  (nstations) int32 0 1 2 3 4 ... 259915 259916 259917 259918 259919
  * time      (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Dimensions without coordinates: nstations
Data variables:
    *empty*
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [ ]:
a["lati"]= list(ds["latitude"].values) * len(list(ds["longitude"].values))
a["long"]= list(ds["longitude"].values) * len(list(ds["latitude"].values))

# a

In [ ]:
a=a.swap_dims({'lati': 'nstations'})
a=a.swap_dims({'long': 'nstations'})



<xarray.Dataset>
Dimensions:   (nstations: 259920, time: 32144)
Coordinates:
    stations  (nstations) int32 0 1 2 3 4 ... 259915 259916 259917 259918 259919
  * time      (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
    lati      (nstations) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
    long      (nstations) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
Dimensions without coordinates: nstations
Data variables:
    *empty*
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [ ]:
a["hs_1_26"]=new_hs_1_26
a

In [133]:
b=len(list(ds["longitude"].values))
b

720

In [136]:
c=list(ds["latitude"].values) * 3
len(c)

1083

In [ ]:
c.

In [23]:
ds.sel({"time":pd.date_range(start='2040-01-01', end='2045-01-01', freq='AS').to_numpy()})

C:\Users\rowe\AppData\Local\Temp\ipykernel_45948\562455148.py:1: FutureWarning: 'AS' is deprecated and will be removed in a future version, please use 'YS' instead.
  ds.sel({"time":pd.date_range(start='2040-01-01', end='2045-01-01', freq='AS').to_numpy()})


<xarray.Dataset>
Dimensions:    (time: 6, latitude: 361, longitude: 720)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 2041-01-01 ... 2045-01-01
Data variables:
    hs_1_26    (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85    (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [10]:
ds.where({"time":pd.date_range(start='2040-01-01', end='2045-01-01', freq='AS').to_numpy()})

C:\Users\rowe\AppData\Local\Temp\ipykernel_45948\2738948212.py:1: FutureWarning: 'AS' is deprecated and will be removed in a future version, please use 'YS' instead.
  ds.where({"time":pd.date_range(start='2040-01-01', end='2045-01-01', freq='AS').to_numpy()})


<xarray.Dataset>
Dimensions:    (latitude: 361, longitude: 720, time: 32144)
Coordinates:
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Data variables:
    *empty*
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [58]:
k=len(ds["latitude"])*len(ds["longitude"])
k
range(k-1)
ds=ds.expand_dims(dim={"nstations": range(len(ds["latitude"])*len(ds["longitude"]))})
ds=ds.rename_vars( {"nstations": "stations" })
ds

<xarray.Dataset>
Dimensions:    (nstations: 259920, time: 32144, latitude: 361, longitude: 720)
Coordinates:
  * stations   (nstations) int32 0 1 2 3 4 ... 259916 259917 259918 259919
  * latitude   (latitude) float32 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * time       (time) datetime64[ns] 2040-01-01 ... 2050-12-31T21:00:00
Dimensions without coordinates: nstations
Data variables:
    hs_1_26    (nstations, time, latitude, longitude) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
    hs_5_85    (nstations, time, latitude, longitude) float32 dask.array<chunksize=(259920, 1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2040-2050 CMIP6 EC-EARTH S...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.204001_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2040-01-01 00:00:00
    stop_date:                     2040-01-31 21:00:00
    westernmost_longitude:         0.

In [42]:
q=ds.sel(nstations=0).compute()
q

<xarray.Dataset>
Dimensions:           (time: 63)
Coordinates:
    continent         |S13 b'South America'
    country           |S5 b'Chile'
    gdp_ppp_cap_3_yr  float64 1.468e+04
    lat               float64 -50.38
    lon               float64 -74.39
  * time              (time) datetime64[ns] 1960-01-01 1961-01-01 ... 2022-01-01
    transect_geom     |S77 b'LINESTRING (-74.3863095545 -50.3776589451, -74.3...
    transect_id       |S13 b'BOX_028_183_0'
Data variables:
    GDP_PPP_Cap       (time) float64 504.8 580.9 663.6 ... 1.624e+04 1.536e+04
Attributes: (12/21)
    AUTHOR:              World Bank
    CITATION:            World Bank. (Year). World Development Indicators. Re...
    COMMENT:             An additional coordinate titled 3 year average is ad...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         GDP per capita is an economic metric that breaks dow...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['GDP per capita', 'economic prosperity', 'national ...
    TEMPORAL_EXTENT:     ['1960-01-01T00:00:00Z', '2022-01-01T00:00:00Z']
    TITLE:               GDP per Capita
    TITLE_ABBREVIATION:  GDPPC
    UNITS:               current US$

In [38]:
nan_indices = np.isnan(ds['lat'].values)
nan_values = ds['lat'][nan_indices].compute()
nan_values

<xarray.DataArray 'lat' (nstations: 0)>
array([], dtype=float64)
Coordinates:
    continent         (nstations) |S23 
    country           (nstations) |S40 
    gdp_ppp_cap_3_yr  (nstations) float64 
    lat               (nstations) float64 
    lon               (nstations) float64 
    transect_geom     (nstations) |S92 
    transect_id       (nstations) |S15 
Dimensions without coordinates: nstations
Attributes:
    long_name:      latitude
    standard_name:  latitude
    units:          degrees_north

In [25]:
from pathlib import Path

filename = "/path/to/myfile.txt"
path = Path(filename)

# Get the stem of the path
file_stem = path.stem

print(file_stem)  

myfile


In [27]:
ds.isnull().compute()

<xarray.Dataset>
Dimensions:           (nstations: 1739919, time: 63)
Coordinates:
    continent         (nstations) |S23 b'South America' ... b'Europe'
    country           (nstations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    gdp_ppp_cap_3_yr  (nstations) float64 1.468e+04 1.468e+04 ... 1.142e+04
    lat               (nstations) float64 -50.38 -50.38 -50.39 ... 64.7 64.7
    lon               (nstations) float64 -74.39 -74.39 -74.38 ... 39.94 39.95
  * time              (time) datetime64[ns] 1960-01-01 1961-01-01 ... 2022-01-01
    transect_geom     (nstations) |S92 b'LINESTRING (-74.3863095545 -50.37765...
    transect_id       (nstations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
Dimensions without coordinates: nstations
Data variables:
    GDP_PPP_Cap       (nstations, time) bool False False False ... False False

In [24]:
pd.date_range(start='1960-01-01T00:00:00.000000000', end='2022-01-01T00:00:00.000000000', freq='YS')
np.arange(1960, 2022 +  1, dtype='int64')

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

In [4]:
ds=xr.open_zarr(r"P:\11208003-latedeo2022\020_InternationalDeltaPortfolio\datasets\00_mapping_global_threat_of_land_subsidence\Global_TLS.zarr")
ds

<xarray.Dataset>
Dimensions:   (stations: 108, time: 2)
Coordinates:
    country   (stations) |S26 dask.array<chunksize=(108,), meta=np.ndarray>
    geometry  (stations) |S1095835 dask.array<chunksize=(1,), meta=np.ndarray>
    lat       (stations) float64 dask.array<chunksize=(108,), meta=np.ndarray>
    lon       (stations) float64 dask.array<chunksize=(108,), meta=np.ndarray>
  * time      (time) int32 2010 2040
Dimensions without coordinates: stations
Data variables:
    eapa      (time, stations) float64 dask.array<chunksize=(2, 108), meta=np.ndarray>
    egdp      (time, stations) float64 dask.array<chunksize=(2, 108), meta=np.ndarray>
    epsi      (time, stations) float64 dask.array<chunksize=(2, 108), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Herrera, G., Ezquerro, P., Tomás, R., Bejar, M., Lop...
    CITATION:            Herrera, G. & Ezquerro, Pablo & Tomás, Roberto & Bej...
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         Subsidence, the lowering of Earth's land surface, is...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['land subsidence', 'groundwater depletion', 'drought']
    TEMPORAL_EXTENT:     ['2010', '2040']
    TITLE:               Land Subsidence Threat
    TITLE_ABBREVIATION:  sub_threat
    UNITS: